# Living Earth - L3 & L4 layers
---
This notebook is aiming to help automatizing the production of Environmental Descriptors (ED) required to produce consistent Level 3+ LCCS maps.

The LivingEarth LCCS code provides an implementation of the Food and Agriculture Organization’s (FAO’s) Land Cover Classification System (LCCS) designed to be applied to Earth Observation (EO) data. The system takes multiple products and combines these through a series of decision trees to produce all classes described as part of LCCS v2, with some modifications where required to apply to EO data.

The Food and Agriculture Organization’s (FAO’s) Land Cover Classification System (LCCS) has two stages, a Dichotomous phase (Level 3) and modular-hierarchical phase (Level 4).

More information can be found at: https://livingearth-lccs.readthedocs.io/en/latest/index.html 

This notebook uses:
- The Google Earth Engine (https://earthengine.google.com) to process satellite data 
- The Coperniucs Land Monitoring Service (CLMS) download API (https://eea.github.io/clms-api-docs/index.html) to download additional environmental layers

Author(s): Carole Planque, Audrey Lambiel, Pablo Timoner, Charlotte Poussin, Gregory Giuliani [UNIGE]
<br>Version: 1.0
<br>Date: 2025-11-11
<br>Supported by: Horizon-Europe LandShift - NEMESIS - MONALISA; Living-Switzerland

---
<b>LCCS Level-3 Layers</b>
1. Initialize and select your sensor (Landsat or Sentinel)
2. Vegetation layer - [vegetat_veg_cat](#vegetat_veg_cat)
3. Water layer - [aquatic_wat_cat](#aquatic_wat_cat)
4. Cultivated layer - [cultman_agr_cat](#cultman_agr_cat)
5. Artificial layer - [artific_urb_cat](#artific_urb_cat)

<b>Additional Layers (L3+)</b>
1. Life Form Layer - [lifeform_veg_cat](#lifeform_veg_cat)
2. Canopy cover - [canopyco_veg_con](#canopyco_veg_con)
3. Leaf Type - [leaftype_veg_cat](#leaftype_veg_cat)

---
## Initialize GEE

In [15]:
import ee
import geemap

In [16]:
Map = geemap.Map()
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topright', transp…

In [17]:
#Config
startDate = '2024-04-01' #TBC
endDate = '2025-10-31' #TBC
site_name = 'switzerland' #TBC
year = '2024' #TBC

In [18]:
#AoI using assets
aoi = ee.FeatureCollection('projects/ee-rs2/assets/switzerland') #TBC
Map.addLayer(aoi,{},'AOI')
Map.centerObject(aoi, 8)

In [ ]:
#Sentinel-2
#Cloud masking 
def cloudMask(image):
    scl = image.select('SCL')
    mask = scl.eq(3).Or(scl.gte(7).And(scl.lte(10)))
    return image.updateMask(mask.eq(0))

#Import Sentinel-2 image collection
CollectionS2 = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED") \
              .filterBounds(aoi) \
              .filterDate(startDate, endDate) \
              .filter(ee.Filter.calendarRange(1, 12, 'month')) \
              .map(cloudMask)

#RGB visualization parameters
visualizationS2 = {
  'bands': ['B4', 'B3', 'B2'],
  'min': 0,
  'max': 1800,
}

#Visualize RGB median
MedianS2 = CollectionS2.median().clip(aoi)
Map.addLayer(MedianS2, visualizationS2, 'Sentinel-2 | True Color')

AttributeError: 'NoneType' object has no attribute 'comm_id'

In [ ]:
# Export to Drive
geemap.ee_export_image_to_drive(
  image=MedianS2, 
  description='RGB_S2_'+site_name+'_'+year+'_Layer',
  fileNamePrefix='RGB_S2_'+site_name+'_'+year,
  folder=site_name,
  region=aoi.geometry(),
  scale=10,
  crs='EPSG:4326',
  maxPixels=1e10
)

---
## Vegetation - vegetat_veg_cat

In [14]:
# Load Sentinel-2 Level-2A image collection
s2 = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED") \
    .filterBounds(aoi) \
    .filterDate(startDate, endDate) \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))

# Function to compute NDVI for each image
def add_ndvi(image):
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    return image.addBands(ndvi)

# Apply NDVI function to each image
s2_ndvi = s2.map(add_ndvi)

# Compute the maximum NDVI composite
max_ndvi = s2_ndvi.qualityMosaic('NDVI').select('NDVI')

# Visualization parameters
vis_params = {
    'min': 0,
    'max': 1,
    'palette': ['red', 'white', 'green']
}

#Visualize
Map.addLayer(max_ndvi.clip(aoi), vis_params, 'Sentinel-2 | maxNDVI')

In [ ]:
# Export to Drive
geemap.ee_export_image_to_drive(
  image=max_ndvi, 
  description='maxNDVI_S2_'+site_name+'_'+year+'_Layer',
  fileNamePrefix='maxNDVI_S2_'+site_name+'_'+year,
  folder=site_name,  
  region=aoi.geometry(),
  scale=10,
  crs='EPSG:4326',
  maxPixels=1e10
)

## Cultivated - cultman_agr_cat

## Water - aquatic_wat_cat

## Artificial - artific_urb_cat

https://land.copernicus.eu/api/en/products/high-resolution-layer-imperviousness
<br>DatasetID - UID: 4354f5ba912d4071b3b5f9300104765b
<br>DatasetDownloadInformationID: f9a55a6d-c98a-4098-bf94-f2ae7cdb0207 [2018]

In [ ]:
import json
import jwt
import time
import requests

base_url = 'https://land.copernicus.eu'

In [ ]:
 # Load saved key from filesystem
service_key = json.load(open('livingearth_gee.json', 'rb'))

private_key = service_key['private_key'].encode('utf-8')

claim_set = {
    "iss": service_key['client_id'],
    "sub": service_key['user_id'],
    "aud": service_key['token_uri'],
    "iat": int(time.time()),
    "exp": int(time.time() + (60 * 60)),
}
grant = jwt.encode(claim_set, private_key, algorithm='RS256')

#Authentication
result = requests.post(
        service_key["token_uri"],
        headers={
            "Accept": "application/json",
            "Content-Type": "application/x-www-form-urlencoded",
        },
        data={
            "grant_type": "urn:ietf:params:oauth:grant-type:jwt-bearer",
            "assertion": grant,
        },
)

access_token_info_json = result.json()
access_token = access_token_info_json.get('access_token')
print(access_token)

In [ ]:
#Download request for a pre-packaged file
#FileID should be changed to download a specific country
url_download_item = f"{base_url}/api/@datarequest_post"
headers = {'Accept': 'application/json', 'Authorization': f'Bearer {access_token}'}
data = {'Datasets': [{'DatasetID': '4354f5ba912d4071b3b5f9300104765b', 'FileID': 'fface0a2-4e04-45ef-aa54-85af8be2a6e6'}]}
response_download_item = requests.post(url_download_item, headers=headers, json=data)
print('Response:',response_download_item)
print('Message with TaskID:\n'+ response_download_item.text)

In [ ]:
#Download request for a extraction by NUTS
#NUTS should be changed to download a specific area
url_download_item = f"{base_url}/api/@datarequest_post"
headers = {'Accept': 'application/json', 'Authorization': f'Bearer {access_token}'}
data = {'Datasets': [{'DatasetID': '4354f5ba912d4071b3b5f9300104765b', 'DatasetDownloadInformationID': '9a55a6d-c98a-4098-bf94-f2ae7cdb0207', 'OutputFormat': 'Geotiff', 'OutputGCS': 'EPSG:4326', 'NUTS': 'ITF5'}]}
response_download_item = requests.post(url_download_item, headers=headers, json=data)
print('Response:',response_download_item)
print('Message with TaskID:\n'+ response_download_item.text)